In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

from data_utils import prep_data



import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random

import os
import codecs

import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

In [2]:
corpus_name = 'data'
corpus = os.path.join(corpus_name)
filename = "formatted_movie_lines.txt"

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\n"


In [3]:
datafile = os.path.join(corpus, filename)

delimiter = str(codecs.decode('\n', "unicode_escape"))

# Initialize lines dict, conversations list, and field ids
lines = {}
conversations = []


In [16]:
MAX_LENGTH = 200  # Maximum sentence length to consider
save_dir = os.path.join("data", "save")
prep_data_obj = prep_data(MAX_LENGTH)
voc, pairs = prep_data_obj.loadPrepareData(corpus, corpus_name, datafile, save_dir)
for pair in pairs[:1]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 442564 sentence pairs
Trimmed to 428758 sentence pairs
Counting words...
Counted words: 33
['canwemakethisquick?roxannekorrineandandrewbarrettarehavinganincrediblyhorrendouspublicbreakuponthequad.again.', 'can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad . again .']


In [17]:
#Count all the characters
all_c = sum(list(voc.word2count.values()))

In [18]:
voc.word2count[' ']/all_c

0.2160765254803255

In [8]:
train_ratio = 0.8
train_pairs = pairs[:int(len(pairs)*train_ratio)]
test_pairs = pairs[int(len(pairs)*train_ratio):]

In [19]:


# Example for validation
small_batch_size = 50
batches = prep_data_obj.batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable.shape)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: torch.Size([143, 50])
lengths: tensor([143, 130, 111, 106, 102,  93,  90,  87,  66,  64,  59,  59,  51,  48,
         45,  42,  40,  38,  38,  34,  33,  27,  26,  24,  23,  22,  21,  21,
         20,  20,  18,  17,  17,  16,  15,  14,  14,  13,  13,  13,  12,  12,
         11,  11,  11,  10,  10,   9,   8,   8])
target_variable: tensor([[13,  4, 11,  ..., 27,  3, 27],
        [ 6,  7, 12,  ...,  8,  4,  4],
        [14,  6,  4,  ..., 18, 25, 18],
        ...,
        [ 6,  0,  0,  ...,  0,  0,  0],
        [17,  0,  0,  ...,  0,  0,  0],
        [ 2,  0,  0,  ...,  0,  0,  0]])
mask: tensor([[ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        ...,
        [ True, False, False,  ..., False, False, False],
        [ True, False, False,  ..., False, False, False],
        [ True, False, False,  ..., False, False, False]])
max_target_len: 189


In [20]:
input_variable[:,0] 

tensor([13, 14, 12, 19, 16, 25, 21, 10,  5, 19,  7, 17, 12, 13, 14,  7, 13, 29,
         8, 21, 19,  5, 11,  8, 23,  8,  5, 10,  5, 19,  7, 28, 27, 18, 19, 22,
         4, 22, 25, 26, 19, 29, 29,  7, 13, 11, 12, 19,  5,  8, 19, 29, 12, 13,
        14, 25, 13, 11, 11, 25,  8, 24, 13, 18, 25, 14, 28, 13, 21, 19,  5, 11,
        10,  5, 19,  7, 28, 13, 11,  8, 25, 25, 26, 19, 16, 13, 29, 12,  8, 21,
        19,  5, 11, 24, 13, 23,  8,  4, 21,  4,  9,  5, 11, 12,  8,  5, 13, 21,
        19,  5, 11, 24, 13, 23,  8,  4, 21,  4,  9,  5, 28,  7, 12, 26, 14, 12,
        19, 16, 25, 21, 13, 10, 13, 25, 25,  9, 26, 14,  8, 25, 29, 17,  2])

In [21]:
target_variable[:, 0]

tensor([13,  6, 14, 12, 19, 16, 25, 21,  6, 10,  5, 19,  7,  6, 17,  6, 12, 13,
        14,  6,  7, 13, 29,  8,  6, 21, 19,  5,  6, 11,  6,  8, 23,  8,  5,  6,
        10,  5, 19,  7,  6, 28,  6, 27, 18, 19, 22,  4, 22, 25, 26,  6, 19, 29,
        29,  6,  7, 13, 11, 12,  6, 19,  5,  8,  6, 19, 29,  6, 12, 13, 14,  6,
        25, 13, 11, 11, 25,  8,  6, 24, 13, 18, 25, 14,  6, 28, 13,  6, 21, 19,
         5,  6, 11,  6, 10,  5, 19,  7,  6, 28,  6, 13,  6, 11,  8, 25, 25,  6,
        26, 19, 16,  6, 13, 29,  6, 12,  8,  6, 21, 19,  5,  6, 11,  6, 24, 13,
        23,  8,  6,  4,  6, 21,  4,  9,  5,  6, 11, 12,  8,  5,  6, 13,  6, 21,
        19,  5,  6, 11,  6, 24, 13, 23,  8,  6,  4,  6, 21,  4,  9,  5,  6, 28,
         7, 12, 26,  6, 14, 12, 19, 16, 25, 21,  6, 13,  6, 10, 13, 25, 25,  6,
         9, 26, 14,  8, 25, 29,  6, 17,  2])

In [22]:
#Input sentence
''.join([voc.index2word[i.item()] for i in input_variable[:,0]])

'ishouldknow?hiswifedontevenknow.probablyoffwithoneofhislittlegirls.idontknow.itellyouifhedontgiveadamnthenidontgiveadamn.whyshouldikillmyself?EOS'

In [23]:
#Output sentence
''.join([voc.index2word[i.item()] for i in target_variable[:,0]])

'i should know ? his wife don t even know . probably off with one of his little girls .i don t know . i tell you if he don t give a damn then i don t give a damn .why should i kill myself ?EOS'

In [24]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [25]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [26]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [27]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [28]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [29]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print(f"total training batches that you will need are {len(training_batches)}")
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [30]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [31]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    #lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Model Output:', ''.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [32]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 512
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = 200
checkpoint_iter = 100
loadFilename = os.path.join(save_dir, model_name, corpus_name,
                           '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                           '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 3.79 GiB total capacity; 0 bytes already allocated; 1.52 GiB free; 0 bytes reserved in total by PyTorch)

In [35]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 200
print_every = 10
save_every = 50

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, train_pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
total training batches that you will need are 200
Initializing ...
Training...
Iteration: 10; Percent complete: 5.0%; Average loss: 0.0720
Iteration: 20; Percent complete: 10.0%; Average loss: 0.0659
Iteration: 30; Percent complete: 15.0%; Average loss: 0.0668
Iteration: 40; Percent complete: 20.0%; Average loss: 0.0719
Iteration: 50; Percent complete: 25.0%; Average loss: 0.0666
Iteration: 60; Percent complete: 30.0%; Average loss: 0.0725
Iteration: 70; Percent complete: 35.0%; Average loss: 0.0668
Iteration: 80; Percent complete: 40.0%; Average loss: 0.0634
Iteration: 90; Percent complete: 45.0%; Average loss: 0.0601
Iteration: 100; Percent complete: 50.0%; Average loss: 0.0668
Iteration: 110; Percent complete: 55.0%; Average loss: 0.0645
Iteration: 120; Percent complete: 60.0%; Average loss: 0.0727
Iteration: 130; Percent complete: 65.0%; Average loss: 0.0685
Iteration: 140; Percent complete: 70.0%; Average loss: 0.0640
Iteration: 150; Perc

In [47]:
b = [i[0] for i in test_pairs[:30] if len(i[0]) > 5]

In [48]:
encoder.eval()
decoder.eval()

searcher = GreedySearchDecoder(encoder, decoder)
try:
    outputs = []
    for input_sentence in b:
        input_sentence = normalizeString(input_sentence)
        output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
        output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
        outputs.append([input_sentence, ''.join(output_words)])
        
except KeyError:
    print("Error: Encountered unknown word.")

In [46]:
outputs

[['whatsgoinon ?', 'what s goin on ? ?'],
 ['rosesaysshesgointohaveababyandshesurelooksitbutidontthinkso .',
  'rose says she s goin to have a baby and she sure looks it but i don t think so . . think so . . think so . . think so . . think so . . think so . .'],
 ['rosesaysshesgointohaveababyandshesurelooksitbutidontthinkso .',
  'rose says she s goin to have a baby and she sure looks it but i don t think so . . think so . . think so . . think so . . think so . . think so . .'],
 ['ababy . . .andsheisntevenmarried .',
  'a baby . . . . and she isn t even married .d she isn t even married .d she isn t even married .d she isn t even married .d she isn t even married .d she isn t even m'],
 ['ababy . . .andsheisntevenmarried .',
  'a baby . . . . and she isn t even married .d she isn t even married .d she isn t even married .d she isn t even married .d she isn t even married .d she isn t even m'],
 ['thatstheleastofhertroubles .rosehadararetropicdiseaseandthelittletubesinherthatbabiesswim

In [ ]:
# test_input = ["thismodelputspacesbetweencharacters",
#               "ithinkicansolvethisriddle",
#               'wellyouareamazing',
#               'adogisverypissedatme!',
#               'ithinkilovethismovie.charactersfromthismotionpictureisawesome',
#               'thisisabeautifulcap',
#               'icansolvethispuzzleveryeasily',
#               'usuallyachairhasfourlegs',
#               'tablealsohasfourlegs',
#               'thisismysignature.',
#               'knowlegeisnotsameaswisdom.',
#               "iamsorryidon'twanttoosoundrudesbutareyouplanningtodothisactivity?"
#               'themorningsunlightgivesyoumorevitamindthantablets'
#              ] 